In [25]:
# Biblioteki
from random import sample
from random import randint

import matplotlib.pyplot as plt
import pandas as pd

In [73]:
# Ustawienia gry - ręczne
liczba_graczy = 4
liczba_kart = 3
minimalne_punkty = 24

In [74]:
# Rodzaje kart
wartosci = range(2,15)
figury = ["kier", "pik", "karo", "trefl"]

# Talia
talia = [(wartosc, figura) for figura in figury for wartosc in wartosci]

# Sprawdzenie
# len(talia) # 52 elementy
# talia # wydrukuj

# Słownik punktów
punkty = {}

for karta in talia:
    if karta[0] <= 10:
        punkty[karta] = karta[0]
    else:
        punkty[karta] = 10

# Sprawdzenie
# punkty

# Ustawienia gry - automatyczne
licznik = []
wylosowane_karty = []
gracze = range(liczba_graczy)

In [75]:
# Wylosuj kartę - funkcja
def losuj_karte(uzyte_karty = wylosowane_karty):
    jest_w_wylosowanych = True
    
    while jest_w_wylosowanych == True:
        id_karty = randint(0, len(talia)-1)
        karta = talia[id_karty]
        jest_w_wylosowanych = id_karty in list(uzyte_karty)
    return(karta, id_karty)

# Sprawdź
# losuj_karte()

def policz_punkty(karty, slownik_punktow = punkty):
    pkt = {}
    
    for karta in karty:
        aktualne_w_kolorze = pkt.get(karta[1], 0)
        aktualne_w_kolorze += slownik_punktow[karta]
        pkt[karta[1]] = aktualne_w_kolorze
    return(pkt)

def wybierz_strategie(pkt):
    inverse = [(value, key) for key, value in pkt.items()]
    return(max(inverse)[1])

def potwierdz_strategie_ten_sam_kolor(karty_gracza):
    stosik = karty_gracza[0]
    karty_gracza.remove(stosik)
    return(stosik, karty_gracza)
    
def potwierdz_strategie_rozne_kolory(karty_gracza, strategia):
    do_usuwania = [(wartosc, kolor) for wartosc,kolor in karty_gracza if kolor != strategia]
    stosik = do_usuwania[0]
    karty_gracza.remove(stosik)
    return(stosik, karty_gracza)

# Sprawdź
# policz_punkty([(8, 'kier'), (3, 'trefl'), (14, 'karo')])
# policz_punkty([(2, 'trefl'), (2, 'kier'), (5, 'kier')])
# wybierz_strategie(policz_punkty([(2, 'trefl'), (2, 'kier'), (5, 'kier')]))
# wybierz_strategie(policz_punkty([(8, 'kier'), (3, 'trefl'), (14, 'karo')]))
# wybierz_strategie(policz_punkty([(8, 'kier'), (10, 'trefl'), (14, 'karo')]))
# a, b = potwierdz_strategie_ten_sam_kolor([(8, 'kier'), (3, 'trefl'), (14, 'karo')])
# a
# b
# potwierdz_strategie_rozne_kolory([(8, 'kier'), (3, 'trefl'), (14, 'karo')], 'trefl')

In [76]:
iteracje = 1000

for i in range(iteracje):
    print("Iteracja", i)
    # Rozpoczęcie gry
    karty_w_rece = {}
    historia_kart = {}
    wylosowane_karty = []

    # Stosik
    wybrana_karta = losuj_karte()
    stosik = wybrana_karta[0]
    wylosowane_karty.append(wybrana_karta[1])
#     print('Na stosiku jest ', stosik)

    # Gracze losują pierwsze karty
    for gracz in gracze:
        # Wylosuj po liczba_kart kart
        karty_gracza = []
        for los in range(liczba_kart):
            wybrana_karta = losuj_karte()
            karty_gracza.append(wybrana_karta[0])
            wylosowane_karty.append(wybrana_karta[1])
        karty_w_rece[gracz] = karty_gracza
        historia_kart[gracz] = [karty_gracza]
#         print("Gracz", gracz, "wylosował karty",karty_w_rece[gracz])

    ###########################

    kontynuuj_gre = True
    do_konca_zostalo_n_ruchow = liczba_graczy - 1
    licznik_rund = 0

    while do_konca_zostalo_n_ruchow > 0:
        licznik_rund += 1
#         print("Runda", licznik_rund)
        for gracz in gracze:
#             print(3*'-', "Gracz", gracz, 3*"-")
            strategia_ind = False 

            # Strategia 1: weź stosik, jeśli zgadza się kolor
#             print("Analiza wzięcia karty ze stosika")
            nowe_karty = karty_w_rece[gracz].copy()
            nowe_karty.append(stosik)
#             print("Karty do rozważenia to", nowe_karty)
            punkty_gracza = policz_punkty(nowe_karty)
            strategia = wybierz_strategie(punkty_gracza)
#             print("Strategia dominująca to", strategia)

            if stosik[1] == strategia:
#                 print("Karta ze stosika jest zgodna z kolorem strategicznym")
                nowe_karty.sort()
                if len(punkty_gracza) == 1: # jeśli wszystkie 1 koloru, wyrzuć najmniejszą
#                     print("Wszystkie karty są tego samego koloru")
                    strategia_ind = True                    
                    stosik, karty_w_rece[gracz] = potwierdz_strategie_ten_sam_kolor(nowe_karty)
                    historia_kart[gracz].append(karty_w_rece[gracz])
#                     print("Nowe karty gracza to", karty_w_rece[gracz])
                else: # jeśli nie wszystkie 1 koloru, wyrzuć najmniejszą koloru niestrategicznego
                    strategia_ind = True
                    stosik, karty_w_rece[gracz] = potwierdz_strategie_rozne_kolory(nowe_karty, strategia)
                    historia_kart[gracz].append(karty_w_rece[gracz])                
#                     print("Nowe karty gracza to", karty_w_rece[gracz])

            # Strategia 2: wylosuj nową kartę
            if strategia_ind == False:
#                 print("Gracz bierze z kupki kart")
                wylosowana_karta = losuj_karte()
                wylosowane_karty.append(wylosowana_karta[1]) # doczep id karty
#                 print("Gracz wylosował", wylosowana_karta[0])

                nowe_karty = karty_w_rece[gracz].copy()
                nowe_karty.append(wylosowana_karta[0])        
                punkty_gracza = policz_punkty(nowe_karty)
                strategia = wybierz_strategie(punkty_gracza)

                if wylosowana_karta[0][1] == strategia: # jeśli pasuje do strategii
#                     print("Karta wylosowana jest zgodna z kolorem strategicznym")
                    nowe_karty.sort()

                    if len(punkty_gracza) == 1: # jeśli wszystkie 1 koloru, wyrzuć najmniejszą
#                         print("Wszystkie karty są tego samego koloru")                   
                        stosik, karty_w_rece[gracz] = potwierdz_strategie_ten_sam_kolor(nowe_karty)
                        historia_kart[gracz].append(karty_w_rece[gracz])
#                         print("Nowe karty gracza to", karty_w_rece[gracz])
                    else: # jeśli nie wszystkie 1 koloru, wyrzuć najmniejszą koloru niestrategicznego
#                         print("Nie wszystkie karty są tego samego koloru")
                        stosik, karty_w_rece[gracz] = potwierdz_strategie_rozne_kolory(nowe_karty, strategia)
                        historia_kart[gracz].append(karty_w_rece[gracz])

                else: # Jeśli nie strategia 2, wyrzuć kartę na stosik
#                     print("Karta wylosowana nie jest zgodna z kolorem strategicznym - idzie na stosik")
                    stosik = wylosowana_karta[0]
                    historia_kart[gracz].append(karty_w_rece[gracz])

            # Odlicz ruch, jeśli jesteśmy w końcowej fazie
            if kontynuuj_gre == False:
                do_konca_zostalo_n_ruchow -= 1
#                 print("Do końca gry pozostało", do_konca_zostalo_n_ruchow, "graczy")

            # Sprawdź, czy kończymy grę
            if kontynuuj_gre == True:
                punkty_gracza = policz_punkty(karty_w_rece[gracz])
                inverse = [(value, key) for key, value in punkty_gracza.items()]
                if max(inverse)[0] > minimalne_punkty:
                    kontynuuj_gre = False
#                     print("Gracz", gracz, "ma", max(inverse)[0], "punktów. Koniec gry za chwilę.")

            # Sprawdź, czy trzeba zmienić talię
            if len(wylosowane_karty) > 50:
                wylosowane_karty = []
                stos_ind = talia.index(stosik)
                wylosowane_karty.append(stos_ind)
                for gracz, karty in karty_w_rece.items():
                    for karta in karty:
                        karta_ind = talia.index(karta)
                        wylosowane_karty.append(karta_ind)

    licznik.append(licznik_rund)

Iteracja 0
Iteracja 1
Iteracja 2
Iteracja 3
Iteracja 4
Iteracja 5
Iteracja 6
Iteracja 7
Iteracja 8
Iteracja 9


KeyboardInterrupt: 

In [ ]:
plt.hist(licznik)

In [ ]:
pd.Series(licznik).value_counts()